In [1]:
from sys import path
path.append(r"../")

dataDir='../Data/Causal/'
storage_path='../Data/'

In [2]:
from utils import Syn_Generator_OOD

Syn_2442 = Syn_Generator_OOD(n=10000, ate=0,sc=1,sh=0,one=1,depX=0.05,depU=0.05,VX=1,mV=2,mX=4,mU=4,mD=2,init_seed=7,seed_coef=10,details=1,storage_path=storage_path)
Syn_2442.run(n=10000, num_reps=10)

SynOOD2_1_0_1_0.05_0.05_1
##############################
The data path is: ../Data//data/SynOOD2_1_0_1_0.05_0.05_1/2_4_4_2/
The ATE:
------------------------------
ate: 1
------------------------------
Next, run dataGenerator: 
Run 0/10. 
Run 1/10. 
Run 2/10. 
Run 3/10. 
Run 4/10. 
Run 5/10. 
Run 6/10. 
Run 7/10. 
Run 8/10. 
Run 9/10. 
------------------------------


In [3]:
Datasets = [Syn_2442]

In [137]:
import sys

import os
import argparse
import pandas as pd
import numpy as np
import torch
sys.path.append(r"../")
sys.path.append('/home/wyliu/code/CB-IV')
from utils import log, CausalDataset
from module.SynCBIV import run as run_SynCBIV

os.environ["CUDA_VISIBLE_DEVICES"] = '1'
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

def get_args():
    argparser = argparse.ArgumentParser(description=__doc__)
    # About run setting !!!!
    argparser.add_argument('--seed',default=2021,type=int,help='The random seed')
    argparser.add_argument('--mode',default='vx',type=str,help='The choice of v/x/vx/xx')
    argparser.add_argument('--rewrite_log',default=False,type=bool,help='Whether rewrite log file')
    argparser.add_argument('--use_gpu',default=False,type=bool,help='The use of GPU')
    # About data setting ~~~~
    argparser.add_argument('--num',default=10000,type=int,help='The num of train\val\test dataset')
    argparser.add_argument('--num_reps',default=10,type=int,help='The num of train\val\test dataset')
    argparser.add_argument('--ate',default=0,type=float,help='The ate of constant')
    argparser.add_argument('--sc',default=1,type=float,help='The sc')
    argparser.add_argument('--sh',default=0,type=float,help='The sh')
    argparser.add_argument('--one',default=1,type=int,help='The dim of Instrumental variables V')
    argparser.add_argument('--depX',default=0.05,type=float,help='Whether generates harder datasets')
    argparser.add_argument('--depU',default=0.05,type=float,help='Whether generates harder datasets')
    argparser.add_argument('--VX',default=1,type=int,help='The dim of Instrumental variables V')
    argparser.add_argument('--mV',default=2,type=int,help='The dim of Instrumental variables V')
    argparser.add_argument('--mX',default=4,type=int,help='The dim of Confounding variables X')
    argparser.add_argument('--mU',default=4,type=int,help='The dim of Unobserved confounding variables U')
    argparser.add_argument('--mD',default=2,type=int,help='The dim of Noise variables D')
    argparser.add_argument('--storage_path',default='../Data/',type=str,help='The dir of data storage')
    # Syn
    argparser.add_argument('--syn_alpha',default=0.01,type=float,help='')
    argparser.add_argument('--syn_lambda',default=0.0001,type=float,help='')
    argparser.add_argument('--syn_twoStage',default=True,type=bool,help='')
    # About Debug or Show
    argparser.add_argument('--verbose',default=1,type=int,help='The level of verbose')
    argparser.add_argument('--epoch_show',default=5,type=int,help='The epochs of show time')
    args = argparser.parse_args(args=[])
    return args

args = get_args()

In [138]:
# set path
which_benchmark = 'SynOOD_'+'_'.join(str(item) for item in [args.sc, args.sh, args.one, args.depX, args.depU,args.VX])
which_dataset = '_'.join(str(item) for item in [args.mV, args.mX, args.mU, args.mD])
resultDir = args.storage_path + f'/results/{which_benchmark}_{which_dataset}/'
dataDir = f'{args.storage_path}/data/{which_benchmark}/{which_dataset}/'
os.makedirs(os.path.dirname(resultDir), exist_ok=True)
logfile = f'{resultDir}/log.txt'
exp = 0

In [139]:
train_df = pd.read_csv(dataDir + f'{exp}/train.csv')

In [140]:
train_df

,v1,v2,x1,x2,x3,x4,u1,u2,u3,u4,d1,d2,z,pi,t,mu0,mu1,y,f
0,-1.471120,-0.111800,-0.295136,0.710984,-0.304386,1.303714,0.148355,1.002048,1.545932,1.429011,1.545932,1.429011,5.895214,0.997255,1.0,0.692565,0.980377,0.984997,0.671763
1,-0.531695,1.523531,-1.159546,0.516137,-1.700485,0.682999,-0.480872,0.664847,-0.689028,0.434670,-0.689028,0.434670,-0.328401,0.418630,0.0,-0.216410,0.788255,-0.217524,0.783802
2,0.643073,-0.619137,-1.318716,-0.010805,-2.002563,0.800997,1.593101,1.186939,0.337385,0.079097,0.337385,0.079097,-0.175905,0.456137,1.0,0.083179,1.307233,1.308752,0.086758
3,-1.347721,0.272128,0.266759,-0.943173,-0.851580,-0.998146,0.569034,-0.258703,0.161201,-0.909284,0.161201,-0.909284,-3.580073,0.027118,0.0,-0.370487,0.490717,-0.361868,0.483942
4,1.636188,-1.386524,-0.354222,0.496414,-1.737128,0.005626,-0.348029,0.436148,0.965803,0.199437,0.965803,0.199437,-1.603814,0.167449,0.0,-0.041994,0.584181,-0.047362,0.598665
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
999995,-0.877484,-1.269841,0.024959,1.117747,1.669038,1.336113,2.622620,-1.101797,1.605368,1.496134,1.605368,1.496134,7.328920,0.999344,1.0,1.096273,2.341073,2.328314,1.076828
999996,1.210811,-1.152097,-0.650039,-0.601750,-0.311533,0.936946,-1.552306,-0.914732,0.082105,-2.323394,0.082105,-2.323394,-5.428502,0.004370,0.0,-0.666838,1.301358,-0.666037,1.312935
999997,-1.055872,0.477439,0.394057,0.903050,0.414196,0.578322,-0.543847,0.752820,-0.871103,-0.048742,-0.871103,-0.048742,1.593830,0.831154,1.0,0.197344,0.387562,0.384767,0.195983
999998,0.671116,-0.370648,-0.835813,-0.508846,-1.245975,0.197627,0.032562,-1.463061,-0.628496,-0.512775,-0.628496,-0.512775,-5.337102,0.004787,0.0,-0.620597,0.668572,-0.607588,0.664995


In [11]:
import os
import argparse
import pandas as pd
import numpy as np
import torch

from utils import log, CausalDataset
from module.Regression import run as run_Reg

os.environ["CUDA_VISIBLE_DEVICES"] = '2'
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

def get_args():
    argparser = argparse.ArgumentParser(description=__doc__)
    # About run setting !!!!
    argparser.add_argument('--seed',default=2021,type=int,help='The random seed')
    argparser.add_argument('--mode',default='vx',type=str,help='The choice of v/x/vx/xx')
    argparser.add_argument('--rewrite_log',default=False,type=bool,help='Whether rewrite log file')
    argparser.add_argument('--use_gpu',default=True,type=bool,help='The use of GPU')
    # About data setting ~~~~
    argparser.add_argument('--num',default=10000,type=int,help='The num of train\val\test dataset')
    argparser.add_argument('--num_reps',default=10,type=int,help='The num of train\val\test dataset')
    argparser.add_argument('--ate',default=0,type=float,help='The ate of constant')
    argparser.add_argument('--mV',default=2,type=int,help='The dim of Instrumental variables V')
    argparser.add_argument('--mX',default=4,type=int,help='The dim of Confounding variables X')
    argparser.add_argument('--mU',default=4,type=int,help='The dim of Unobserved confounding variables U')
    argparser.add_argument('--mD',default=2,type=int,help='The dim of Noise variables D')
    argparser.add_argument('--storage_path',default='../Data/',type=str,help='The dir of data storage')
    # About Regression_t
    argparser.add_argument('--regt_batch_size',default=500,type=int,help='The size of one batch')
    argparser.add_argument('--regt_lr',default=0.05,type=float,help='The learning rate')
    argparser.add_argument('--regt_num_epoch',default=3,type=int,help='The num of total epoch')
    args = argparser.parse_args(args=[])
    return args

args = get_args()

if args.use_gpu:
    device = torch.device('cuda' if torch.cuda.is_available() and args.use_gpu else "cpu")

In [16]:
# run vx
for data in Datasets:
    which_benchmark = data.which_benchmark
    which_dataset = data.which_dataset
    args.num_reps = data.num_reps
    args.mV = data.mV
    args.mX = data.mX
    args.mU = data.mU
    args.mD = data.mD
    args.mode = 'vx'

    resultDir = args.storage_path + f'/results/{which_benchmark}_{which_dataset}/'
    dataDir = f'{args.storage_path}/data/{which_benchmark}/{which_dataset}/'
    os.makedirs(os.path.dirname(resultDir), exist_ok=True)
    logfile = f'{resultDir}/log.txt'

    if args.rewrite_log:
        f = open(logfile,'w')
        f.close()

    for exp in range(args.num_reps):
        train_df = pd.read_csv(dataDir + f'{exp}/train.csv')
        val_df = pd.read_csv(dataDir + f'{exp}/val.csv')
        test_df = pd.read_csv(dataDir + f'{exp}/test.csv')
                                                    
        train = CausalDataset(train_df, variables = ['u','x','v','d','z','p','s','m','t','g','y','f','c'])
        val = CausalDataset(val_df, variables = ['u','x','v','d','z','p','s','m','t','g','y','f','c'])
        test = CausalDataset(test_df, variables = ['u','x','v','d','z','p','s','m','t','g','y','f','c'])

        train,val,test = run_Reg(exp, args, dataDir, resultDir, train, val, test, device)

Exp 0 :this is the 0/3 epochs.
The train accuracy: 74.18 %
The test  accuracy: 74.65 %
Exp 0 :this is the 1/3 epochs.
The train accuracy: 74.89 %
The test  accuracy: 75.01 %
Exp 0 :this is the 2/3 epochs.
The train accuracy: 75.00 %
The test  accuracy: 74.99 %
Exp 1 :this is the 0/3 epochs.
The train accuracy: 74.30 %
The test  accuracy: 74.16 %
Exp 1 :this is the 1/3 epochs.
The train accuracy: 74.84 %
The test  accuracy: 74.37 %
Exp 1 :this is the 2/3 epochs.
The train accuracy: 74.78 %
The test  accuracy: 74.55 %
Exp 2 :this is the 0/3 epochs.
The train accuracy: 73.59 %
The test  accuracy: 74.15 %
Exp 2 :this is the 1/3 epochs.
The train accuracy: 73.92 %
The test  accuracy: 74.55 %
Exp 2 :this is the 2/3 epochs.
The train accuracy: 74.04 %
The test  accuracy: 74.47 %
Exp 3 :this is the 0/3 epochs.
The train accuracy: 74.55 %
The test  accuracy: 75.37 %
Exp 3 :this is the 1/3 epochs.
The train accuracy: 75.07 %
The test  accuracy: 75.77 %
Exp 3 :this is the 2/3 epochs.
The train ac

In [17]:
train_df = pd.read_csv(dataDir + f'{exp}/{args.mode}/train.csv')

In [18]:
pd.set_option("display.max_columns",30)
train_df

,u0,u1,u2,u3,x0,x1,x2,x3,v0,v1,d0,d1,z0,p0,s0,m0,m1,t0,y0,f0,c0,c1,c2,c3,c4,c5
0,0.167895,-1.156714,0.286217,-2.299522,3.342740,0.827564,0.782110,0.063012,-1.001001,-0.156685,0.286217,-2.299522,-1.462451,0.188093,0.816494,0.251663,2.401292,1.0,2.406276,0.262091,-1.001001,-0.156685,3.342740,0.827564,0.782110,0.063012
1,0.767891,-0.756043,0.832192,0.596024,-0.171198,-0.302362,0.772823,0.532838,-0.071288,0.133359,0.832192,0.596024,2.244047,0.904136,0.736478,0.284021,0.401369,1.0,0.409404,0.295168,-0.071288,0.133359,-0.171198,-0.302362,0.772823,0.532838
2,0.604969,-1.128767,-0.090741,-0.994492,1.590143,0.030578,0.864707,-1.346071,-2.587700,0.083583,-0.090741,-0.994492,-4.581932,0.010131,0.295385,-0.058709,0.965808,0.0,-0.065913,0.956466,-2.587700,0.083583,1.590143,0.030578,0.864707,-1.346071
3,-0.315662,-0.436604,-0.240507,-0.022828,-0.545619,1.983291,0.015091,-1.760418,0.441269,0.780610,-0.240507,-0.022828,-0.015842,0.496040,0.493013,-0.165407,0.959884,0.0,-0.166347,0.955113,0.441269,0.780610,-0.545619,1.983291,0.015091,-1.760418
4,-0.266820,-0.010214,-1.328231,0.447407,0.409477,0.515354,0.736945,-0.872909,-0.317921,1.044166,-1.328231,0.447407,0.038942,0.509734,0.682651,-0.046124,0.471748,0.0,-0.061809,0.476882,-0.317921,1.044166,0.409477,0.515354,0.736945,-0.872909
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,0.880253,-1.033163,-1.045848,1.944765,-0.121837,-0.479285,1.424171,0.068950,0.625478,1.723168,-1.045848,1.944765,0.735910,0.676101,0.529990,0.204751,1.124469,1.0,1.120370,0.222028,0.625478,1.723168,-0.121837,-0.479285,1.424171,0.068950
9996,2.053585,0.988764,1.191509,-0.024040,1.080524,-1.805821,-0.002919,0.264813,1.502980,-0.922656,1.191509,-0.024040,7.036573,0.999122,0.814706,0.468302,1.389224,1.0,1.401081,0.483496,1.502980,-0.922656,1.080524,-1.805821,-0.002919,0.264813
9997,-1.689446,-0.428585,0.918160,0.333585,-0.860455,-0.612786,-1.377271,-2.129401,0.520946,-0.460582,0.918160,0.333585,-6.012211,0.002443,0.033130,-0.730775,1.442416,0.0,-0.756161,1.428220,0.520946,-0.460582,-0.860455,-0.612786,-1.377271,-2.129401
9998,1.098371,-1.140961,1.265852,-2.064149,-1.214361,-1.626790,-1.261606,-0.338284,2.029131,-1.239294,1.265852,-2.064149,-5.729954,0.003237,0.045546,-0.660241,1.774814,0.0,-0.651293,1.764046,2.029131,-1.239294,-1.214361,-1.626790,-1.261606,-0.338284


In [70]:
data = train_df
r = -3.0
n = 100
dim_v = 2

In [71]:
type(data)

pandas.core.frame.DataFrame

In [72]:
nall = data.shape[0]
prob = np.ones(nall)

ite = data['m1']-data['m0']

In [73]:
idv = 0
data[f'd{idv}']

0       0.286217
1       0.832192
2      -0.090741
3      -0.240507
4      -1.328231
          ...   
9995   -1.045848
9996    1.191509
9997    0.918160
9998    1.265852
9999   -0.701346
Name: d0, Length: 10000, dtype: float64

In [103]:
if r!=0.0:
    for idv in range(dim_v):
        # 和表达式不一样
        d = np.abs(data[f'd{idv}'] - np.sign(r) * ite) 
        prob = prob * np.power(np.abs(r), -10 * d)
prob = prob / np.sum(prob)
idx = np.random.choice(range(nall), n, p=prob, replace=False)

In [113]:
new_data = data.iloc[idx].reset_index(drop=True)

In [115]:
# x = data['x'][idx, :]
t = new_data['t0']
mu0 = new_data['m0']
mu1 = new_data['m1']

In [116]:
new_data

,u0,u1,u2,u3,x0,x1,x2,x3,v0,v1,d0,d1,z0,p0,s0,m0,m1,t0,y0,f0,c0,c1,c2,c3,c4,c5
0,1.007056,-0.894096,-0.324679,-0.313057,0.654318,1.209629,0.245933,0.836097,1.228454,-2.021394,-0.324679,-0.313057,0.779864,0.685651,0.776961,0.302650,0.583483,1.0,0.582199,0.302440,1.228454,-2.021394,0.654318,1.209629,0.245933,0.836097
1,0.214399,0.163062,-1.600483,-1.565804,-1.642831,-0.183210,1.155825,0.143346,-1.212817,-0.152979,-1.600483,-1.565804,-1.295215,0.214971,0.702357,-0.414462,1.146848,0.0,-0.417533,1.160586,-1.212817,-0.152979,-1.642831,-0.183210,1.155825,0.143346
2,0.336961,-0.741892,-0.905139,-0.894536,-0.557545,0.550024,-0.968753,0.938616,-0.287662,0.692125,-0.905139,-0.894536,-1.701195,0.154309,0.695990,-0.280283,0.589535,0.0,-0.293789,0.598937,-0.287662,0.692125,-0.557545,0.550024,-0.968753,0.938616
3,-0.469040,1.614628,-0.496621,-0.465961,-0.099751,-0.262507,0.940632,-0.324690,-1.539903,0.731642,-0.496621,-0.465961,0.398235,0.598263,0.479338,0.054586,0.544981,0.0,0.062497,0.541106,-1.539903,0.731642,-0.099751,-0.262507,0.940632,-0.324690
4,0.086790,1.831792,-1.181609,-1.191787,1.097338,-0.753177,0.115052,-0.872587,0.717865,-0.569546,-1.181609,-1.191787,0.348522,0.586259,0.685826,-0.108523,1.090703,1.0,1.094935,-0.116035,0.717865,-0.569546,1.097338,-0.753177,0.115052,-0.872587
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,-0.548173,1.210459,-0.391798,-0.089041,1.328257,-0.240506,0.419812,-0.025410,-1.100614,-0.950543,-0.391798,-0.089041,0.430312,0.605948,0.565868,0.207950,0.490767,1.0,0.480458,0.198450,-1.100614,-0.950543,1.328257,-0.240506,0.419812,-0.025410
96,0.794077,-0.480264,-0.622299,-0.893365,1.230082,-1.193370,-0.107460,0.161166,0.387600,-1.446245,-0.622299,-0.893365,1.091252,0.748618,0.816549,-0.138929,0.627665,1.0,0.623764,-0.137800,0.387600,-1.446245,1.230082,-1.193370,-0.107460,0.161166
97,0.529162,0.255169,-0.564215,-0.602266,-1.075038,-0.141032,-0.261335,-0.776198,-2.142200,0.104510,-0.564215,-0.602266,-0.347546,0.413978,0.425635,-0.329469,0.359070,0.0,-0.317411,0.377362,-2.142200,0.104510,-1.075038,-0.141032,-0.261335,-0.776198
98,0.309597,0.421969,-0.150713,-0.374617,1.287842,2.104527,0.110081,-0.131788,0.395663,-0.396234,-0.150713,-0.374617,3.252565,0.962765,0.816378,0.447112,0.819252,1.0,0.798962,0.442382,0.395663,-0.396234,1.287842,2.104527,0.110081,-0.131788


In [117]:
# continuous y
y0_cont = mu0 + np.random.normal(loc=0., scale=.1, size=n)
y1_cont = mu1 + np.random.normal(loc=0., scale=.1, size=n)



In [118]:
y0_cont

0     0.205690
1    -0.469265
2    -0.328198
3     0.173589
4    -0.106443
        ...   
95    0.262732
96   -0.227543
97   -0.297908
98    0.534300
99   -0.037485
Name: m0, Length: 100, dtype: float64

In [122]:
yf_cont, ycf_cont = pd.Series(np.zeros(n), dtype=float), pd.Series(np.zeros(n), dtype=float)
yf_cont[t>0], yf_cont[t<1] = y1_cont[t>0], y0_cont[t<1]
ycf_cont[t>0], ycf_cont[t<1] = y0_cont[t>0], y1_cont[t<1]

In [124]:
yf_cont

0     0.575656
1    -0.469265
2    -0.328198
3     0.173589
4     0.968870
        ...   
95    0.360613
96    0.449222
97   -0.297908
98    0.859681
99    0.533304
Length: 100, dtype: float64

In [131]:

'''
description: 
param {*} data 输入数据
param {*} r bias rate 
param {*} n size
param {*} dim_v Dimensions of irrelevant variables
return {*}
'''
def correlation_sample(data, r, n, dim_d):
    
    nall = data.shape[0]
    prob = np.ones(nall)

    ite = data['m1']-data['m0']

    if r!=0.0:
        for idv in range(dim_d):
            # 和表达式不一样
            d = np.abs(data[f'd{idv}'] - np.sign(r) * ite) 
            prob = prob * np.power(np.abs(r), -10 * d)
    prob = prob / np.sum(prob)
    idx = np.random.choice(range(nall), n, p=prob, replace=False)

    new_data = data.iloc[idx].reset_index(drop=True)
    t = new_data['t0']
    mu0 = new_data['m0']
    mu1 = new_data['m1']

    # continuous y
    y0_cont = mu0 + np.random.normal(loc=0., scale=.1, size=n)
    y1_cont = mu1 + np.random.normal(loc=0., scale=.1, size=n)

    yf_cont, ycf_cont = pd.Series(np.zeros(n), dtype=float), pd.Series(np.zeros(n), dtype=float)
    yf_cont[t>0], yf_cont[t<1] = y1_cont[t>0], y0_cont[t<1]
    ycf_cont[t>0], ycf_cont[t<1] = y0_cont[t>0], y1_cont[t<1]

    new_data['m0'] = y0_cont
    new_data['m1'] = y1_cont
    new_data['y0'] = yf_cont
    new_data['f0'] = ycf_cont

    # binary y
    # median_0 = np.median(mu0)
    # median_1 = np.median(mu1)
    # mu0[mu0 >= median_0] = 1.
    # mu0[mu0 < median_0] = 0.
    # mu1[mu1 < median_1] = 0.
    # mu1[mu1 >= median_1] = 1.

    # yf_bin, ycf_bin = np.zeros(n), np.zeros(n)
    # yf_bin[t>0], yf_bin[t<1] = mu1[t>0], mu0[t<1]
    # ycf_bin[t>0], ycf_bin[t<1] = mu0[t>0], mu1[t<1]

    return new_data

In [132]:
new_data = correlation_sample(data=data, r=-3.0, n=1000, dim_d=2)

In [133]:
new_data

,u0,u1,u2,u3,x0,x1,x2,x3,v0,v1,d0,d1,z0,p0,s0,m0,m1,t0,y0,f0,c0,c1,c2,c3,c4,c5
0,0.412597,1.015732,-0.372242,-0.544525,1.502877,-0.042292,-0.971592,0.945440,-1.116039,0.199967,-0.372242,-0.544525,0.260268,0.564702,0.526119,0.117756,0.826064,0.0,0.117756,0.826064,-1.116039,0.199967,1.502877,-0.042292,-0.971592,0.945440
1,-0.158368,1.531721,-0.392341,-0.462790,-0.446857,-0.433481,-0.104351,-0.715363,-0.397650,0.310295,-0.392341,-0.462790,-1.138644,0.242569,0.188760,-0.185488,0.353624,1.0,0.353624,-0.185488,-0.397650,0.310295,-0.446857,-0.433481,-0.104351,-0.715363
2,-0.431554,0.833924,-0.786128,-0.545164,0.830690,-1.452424,1.116862,0.707048,2.113583,1.954535,-0.786128,-0.545164,-0.809827,0.307927,0.599672,-0.057987,0.769417,1.0,0.769417,-0.057987,2.113583,1.954535,0.830690,-1.452424,1.116862,0.707048
3,1.408007,-0.625117,-0.515285,-0.689212,-0.576736,1.543412,1.684184,0.540603,-1.400080,0.034899,-0.515285,-0.689212,3.631196,0.974199,0.901739,0.472281,1.159678,1.0,1.159678,0.472281,-1.400080,0.034899,-0.576736,1.543412,1.684184,0.540603
4,0.189132,1.598882,-0.638602,-0.631110,0.089298,-0.699753,1.925159,0.164783,0.239183,0.591193,-0.638602,-0.631110,1.605458,0.832780,0.726518,0.247283,1.070420,1.0,1.070420,0.247283,0.239183,0.591193,0.089298,-0.699753,1.925159,0.164783
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,0.738708,0.882093,-1.025918,-0.283521,0.772176,-0.566353,0.152319,0.116829,0.724805,-1.223772,-1.025918,-0.283521,2.039098,0.884841,0.816985,0.180780,0.478646,0.0,0.180780,0.478646,0.724805,-1.223772,0.772176,-0.566353,0.152319,0.116829
996,0.441568,-0.843707,-0.779115,-0.815847,-0.156617,0.788478,0.238006,0.713080,0.260122,0.715827,-0.779115,-0.815847,0.109519,0.527353,0.832451,-0.263345,0.286674,0.0,-0.263345,0.286674,0.260122,0.715827,-0.156617,0.788478,0.238006,0.713080
997,1.219935,-0.298843,-1.196468,-1.426740,0.541414,-0.237146,-1.469833,-1.152667,1.057478,-0.896438,-1.196468,-1.426740,-3.235227,0.037861,0.214769,-0.460109,1.106488,1.0,1.106488,-0.460109,1.057478,-0.896438,0.541414,-0.237146,-1.469833,-1.152667
998,0.471752,0.911672,-0.907880,-1.026951,0.827896,-1.114166,0.239738,0.625815,0.030701,2.228424,-0.907880,-1.026951,-2.429540,0.080948,0.273022,0.036700,0.624979,0.0,0.036700,0.624979,0.030701,2.228424,0.827896,-1.114166,0.239738,0.625815


In [130]:
data

,u0,u1,u2,u3,x0,x1,x2,x3,v0,v1,d0,d1,z0,p0,s0,m0,m1,t0,y0,f0,c0,c1,c2,c3,c4,c5
0,0.167895,-1.156714,0.286217,-2.299522,3.342740,0.827564,0.782110,0.063012,-1.001001,-0.156685,0.286217,-2.299522,-1.462451,0.188093,0.816494,0.251663,2.401292,1.0,2.406276,0.262091,-1.001001,-0.156685,3.342740,0.827564,0.782110,0.063012
1,0.767891,-0.756043,0.832192,0.596024,-0.171198,-0.302362,0.772823,0.532838,-0.071288,0.133359,0.832192,0.596024,2.244047,0.904136,0.736478,0.284021,0.401369,1.0,0.409404,0.295168,-0.071288,0.133359,-0.171198,-0.302362,0.772823,0.532838
2,0.604969,-1.128767,-0.090741,-0.994492,1.590143,0.030578,0.864707,-1.346071,-2.587700,0.083583,-0.090741,-0.994492,-4.581932,0.010131,0.295385,-0.058709,0.965808,0.0,-0.065913,0.956466,-2.587700,0.083583,1.590143,0.030578,0.864707,-1.346071
3,-0.315662,-0.436604,-0.240507,-0.022828,-0.545619,1.983291,0.015091,-1.760418,0.441269,0.780610,-0.240507,-0.022828,-0.015842,0.496040,0.493013,-0.165407,0.959884,0.0,-0.166347,0.955113,0.441269,0.780610,-0.545619,1.983291,0.015091,-1.760418
4,-0.266820,-0.010214,-1.328231,0.447407,0.409477,0.515354,0.736945,-0.872909,-0.317921,1.044166,-1.328231,0.447407,0.038942,0.509734,0.682651,-0.046124,0.471748,0.0,-0.061809,0.476882,-0.317921,1.044166,0.409477,0.515354,0.736945,-0.872909
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,0.880253,-1.033163,-1.045848,1.944765,-0.121837,-0.479285,1.424171,0.068950,0.625478,1.723168,-1.045848,1.944765,0.735910,0.676101,0.529990,0.204751,1.124469,1.0,1.120370,0.222028,0.625478,1.723168,-0.121837,-0.479285,1.424171,0.068950
9996,2.053585,0.988764,1.191509,-0.024040,1.080524,-1.805821,-0.002919,0.264813,1.502980,-0.922656,1.191509,-0.024040,7.036573,0.999122,0.814706,0.468302,1.389224,1.0,1.401081,0.483496,1.502980,-0.922656,1.080524,-1.805821,-0.002919,0.264813
9997,-1.689446,-0.428585,0.918160,0.333585,-0.860455,-0.612786,-1.377271,-2.129401,0.520946,-0.460582,0.918160,0.333585,-6.012211,0.002443,0.033130,-0.730775,1.442416,0.0,-0.756161,1.428220,0.520946,-0.460582,-0.860455,-0.612786,-1.377271,-2.129401
9998,1.098371,-1.140961,1.265852,-2.064149,-1.214361,-1.626790,-1.261606,-0.338284,2.029131,-1.239294,1.265852,-2.064149,-5.729954,0.003237,0.045546,-0.660241,1.774814,0.0,-0.651293,1.764046,2.029131,-1.239294,-1.214361,-1.626790,-1.261606,-0.338284
